<a href="https://colab.research.google.com/github/henokzion/workout_performance_tracker/blob/main/workout_perfomance_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup credentials for readding data from google sheets

In [75]:
!pip install --upgrade gspread

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

Import modules for data processing

In [76]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Read data : Currenlty my workout split is, a mix between push/pull/logs and upper/lower


In [77]:
spreadsheet_name = 'Workout LPPLU'
leg_day = 'Legs'
push_day = 'Push'
pull_day = 'Pull'
lower_body_day = 'Lower body'
upper_body_day = 'Upper body'

spreadsheet = gc.open(spreadsheet_name)

def read_worksheet(worksheet_name):
  try:
    worksheet = gc.open(spreadsheet_name).worksheet(worksheet_name)
    # get_all_values gives a list of rows.
    rows = worksheet.get_all_values()
    return pd.DataFrame.from_records(rows)
  except gspread.SpreadsheetNotFound:
    print(f"Spreadsheet '{spreadsheet_name}' not found.")
    return None
  except gspread.WorksheetNotFound:
    print(f"Worksheet '{worksheet_name}' not found in spreadsheet '{spreadsheet_name}'.")
    return None
  except Exception as e:
    print(f"An error occurred: {e}")

leg_day_df = read_worksheet(leg_day)
push_day_df = read_worksheet(push_day)
pull_day_df = read_worksheet(pull_day)
lower_body_day_df = read_worksheet(lower_body_day)
upper_body_day_df = read_worksheet(upper_body_day)


# Leg day

In [78]:
structured_data = []  # Store transformed workout data
session_counter = 1

leg_day_df.columns = leg_day_df.iloc[0]
# Rename first column (which is empty in some sheets)
leg_day_df.rename(columns={leg_day_df.columns[0]: 'PlaceHolder'}, inplace=True)

# Extract muscle target (first row) and number of sets (second row)
muscle_targets = leg_day_df.iloc[1]  # First row contains muscle targets
num_sets = leg_day_df.iloc[2]        # Second row contains number of sets

# Remove first two rows before reshaping
df_cleaned = leg_day_df.iloc[3:, 1:].reset_index(drop=True)
df_cleaned.insert(0, 'Workout Session', range(1, len(df_cleaned) + 1))


# Melt (reshape) from wide to long format
df_melted = df_cleaned.melt(id_vars=['Workout Session'], var_name='Exercise', value_name='Performance')

# Assign muscle target and number of sets based on column names
df_melted['Muscle Target'] = df_melted['Exercise'].map(muscle_targets)
df_melted['Number of Sets'] = df_melted['Exercise'].map(num_sets).astype(float)

# Assign the split (Legs, Pull, Push, etc.)
df_melted['Split'] = "leg day"


df_melted.head()

,Workout Session,Exercise,Performance,Muscle Target,Number of Sets,Split
0,1,Lying Leg curl,"28,12",Hamstring,3.0,leg day
1,2,Lying Leg curl,,Hamstring,3.0,leg day
2,3,Lying Leg curl,"28,12",Hamstring,3.0,leg day
3,4,Lying Leg curl,"35,8",Hamstring,3.0,leg day
4,5,Lying Leg curl,"35,8",Hamstring,3.0,leg day
